### Saved Model - Format

In [ ]:
#!pip install pyyaml h5py

In [1]:
import os
import sys
import yaml
import tensorflow as tf
from tensorflow import keras

2024-04-23 11:04:58.839159: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 11:04:58.839184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 11:04:58.840245: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 11:04:58.846006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 11:04:59.584063: W tensorflow/compiler/tf2

In [3]:
from model.dataset import Dataset
from model.fp.nnfp import get_fingerprinter
from model.fp.NTxent_loss_single_gpu import NTxentLoss

In [4]:
def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [5]:
config:str = "default" 
cfg = load_config(config)

cli: Configuration from ./config/default.yaml


In [6]:
if cfg['LOSS']['LOSS_MODE'].upper() == 'NTXENT': # Default
    loss_obj_train = NTxentLoss(
        n_org=cfg['BSZ']['TR_N_ANCHOR'],
        n_rep=cfg['BSZ']['TR_BATCH_SZ'] - cfg['BSZ']['TR_N_ANCHOR'],
        tau=cfg['LOSS']['TAU'])

2024-04-23 11:05:09.304842: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-23 11:05:09.305028: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-23 11:05:09.340129: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
from model.trainer import build_fp

In [9]:
from model.fp.melspec.melspectrogram import get_melspec_layer
from model.fp.specaug_chain.specaug_chain import get_specaug_chain_layer

In [10]:
def build_fp(cfg):
    """ Build fingerprinter """
    # m_pre: log-power-Mel-spectrogram layer, S.
    m_pre = get_melspec_layer(cfg, trainable=False)

    # m_specaug: spec-augmentation layer.
    m_specaug = get_specaug_chain_layer(cfg, trainable=False)
    assert(m_specaug.bypass==False) # Detachable by setting m_specaug.bypass.

    # m_fp: fingerprinter g(f(.)).
    m_fp = get_fingerprinter(cfg, trainable=False)
    #return m_pre, m_specaug, m_fp
    return m_pre, m_specaug, m_fp

In [11]:
m_pre, m_specaug, m_fp = build_fp(cfg)

In [12]:
dataset = Dataset(cfg)

In [43]:
lr_schedule = tf.keras.experimental.CosineDecay(
            initial_learning_rate=float(cfg['TRAIN']['LR']),
            decay_steps=1,
            alpha=1e-06)

In [44]:
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [45]:
m_fp.compile(optimizer=opt,
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=[tf.metrics.SparseCategoricalAccuracy()])

In [ ]:
m_fp.summary()

In [ ]:
tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/')

In [19]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [38]:
import numpy as np

In [37]:
input_1s = tf.constant(np.random.randn(3,256,32,1), dtype=tf.float32) # BxFxTx1
fprinter = FingerPrinter(emb_sz=128, fc_unit_dim=[32, 1], norm='layer_norm2d')
emb_1s = fprinter(input_1s) # BxD

NameError: name 'np' is not defined

In [36]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Defina a estrutura do seu modelo
# Exemplo:
# model = create_model()
model=m_fp
input_shape = (256,32,1)

model.build((None,) + input_shape)

# Carregar os arquivos de checkpoint usando tf.train.Checkpoint
checkpoint = tf.train.Checkpoint(model=model)
#checkpoint.restore(tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/'))
checkpoint.restore('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/ckpt-100.index')

# Obtendo o modelo do checkpoint
model = checkpoint.model

# Salvar o modelo com os pesos no formato *.pb
tf.saved_model.save(model, '/home/rodrigo/Documents/neural-audio-fp/guardarCheks/')

ValueError: Weights for model 'sequential_9' have not yet been created. Weights are created when the model is first called on inputs or `build()` is called with an `input_shape`.

In [ ]:
# Carregar o último ponto de verificação (checkpoint)
latest_checkpoint = tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/')
model = load_model(latest_checkpoint)

# Salvar o modelo com os pesos no formato *.pb
tf.saved_model.save(model, '/home/rodrigo/Documents/neural-audio-fp/guardarCheks/')

In [17]:
checkpoint_dir = '/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/'
os.listdir(checkpoint_dir)

['ckpt-35.data-00000-of-00001',
 'checkpoint',
 'ckpt-67.data-00000-of-00001',
 'ckpt-35.index',
 'ckpt-59.index',
 'ckpt-55.data-00000-of-00001',
 'ckpt-95.data-00000-of-00001',
 'ckpt-75.data-00000-of-00001',
 'ckpt-100.index',
 'ckpt-3.index',
 'ckpt-91.index',
 'ckpt-79.index',
 'ckpt-59.data-00000-of-00001',
 'ckpt-99.data-00000-of-00001',
 'ckpt-99.index',
 'ckpt-3.data-00000-of-00001',
 'ckpt-87.data-00000-of-00001',
 'ckpt-39.index',
 'ckpt-27.data-00000-of-00001',
 'ckpt-15.index',
 'ckpt-27.index',
 'ckpt-71.index',
 'ckpt-15.data-00000-of-00001',
 'ckpt-19.data-00000-of-00001',
 'ckpt-87.index',
 'ckpt-51.index',
 'ckpt-91.data-00000-of-00001',
 'ckpt-51.data-00000-of-00001',
 'ckpt-98.index',
 'ckpt-100.data-00000-of-00001',
 'ckpt-95.index',
 'ckpt-98.data-00000-of-00001',
 'ckpt-83.data-00000-of-00001',
 'ckpt-75.index',
 'ckpt-47.index',
 'ckpt-79.data-00000-of-00001',
 'ckpt-83.index',
 'ckpt-67.index',
 'ckpt-43.data-00000-of-00001',
 'ckpt-43.index',
 'ckpt-23.index',

In [7]:
latest = tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/')

In [8]:
model = get_fingerprinter(cfg, trainable=True)

In [47]:
dataset = Dataset(cfg)
train_ds = dataset.get_train_ds(cfg['DATA_SEL']['REDUCE_ITEMS_P'])

In [51]:
model.compile(optimizer='ADAM', loss=loss_obj_train, metrics=['accuracy', 'loss_weights='])

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

In [9]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model.load_weights(latest)